In [4]:
library("knitr")
library("glmnet")
library("ggplot2")
library("tidyverse")
library("tidymodels")
library("data.table")

## Regression

Here we use the `bats` dataset to build a regression model using the programming framework provided by the R package `tidymodels`.

In [5]:
ch4 <- readxl::read_excel("../data/DNA methylation data.xlsm", sheet = 1)
ch4 <- na.omit(ch4[,-c(1,3)])
head(ch4)

Age,CpG 1 TET2,CpG 2 TET2,CpG 3 TET2,CpG 4 TET2,CpG GRIA2 1,CpG GRIA2 2,ASPA 1
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,29,21,26,31,2,2,61
0,30,21,24,32,1,2,59
1,44,38,51,53,1,5,48
1,48,36,50,46,1,2,53
1,22,9,28,31,2,2,41
1,42,30,40,41,0,4,59


#### 1. partitioning the data

We now use `k-fold` cross-validation to estimate the performance of our linear regresdsion model for the prediction of the age of bats based on epigenetics data.
The dataset is small, and therefore we apply 5-fold cross-validation.

The objective is to get a **robust estimate** of the model **predictive ability**.

In [6]:
folds <- vfold_cv(ch4, v = 5)
folds

splits,id
<list>,<chr>
"0 , 0 , 1 , 1 , 1 , 1 , 1 , 2 , 2 , 2 , 2 , 2 , 2 , 2 , 3 , 3 , 3 , 3 , 4 , 4 , 4 , 4 , 4 , 4 , 4 , 5 , 5 , 5 , 5 , 5 , 5 , 5 , 5 , 5 , 5 , 6 , 6 , 6 , 6 , 6 , 6 , 6 , 7 , 7 , 7 , 8 , 8 , 8 , 9 , 9 , 10 , 11 , 11 , 11 , 12 , 12 , 12 , 14 , 14 , 29 , 30 , 44 , 48 , 22 , 42 , 47 , 61 , 50 , 53 , 60 , 53 , 49 , 57 , 52 , 50 , 69 , 53 , 64 , 69 , 61 , 59 , 52 , 55 , 59 , 67 , 54 , 65 , 51 , 57 , 58 , 62 , 70 , 47 , 66 , 38 , 43 , 56 , 51 , 55 , 66 , 61 , 61 , 61 , 61 , 67 , 68 , 63 , 68 , 64 , 63 , 57 , 67 , 68 , 50 , 59 , 65 , 71 , 62 , 21 , 21 , 38 , 36 , 9 , 30 , 43 , 47 , 42 , 41 , 46 , 41 , 39 , 42 , 42 , 42 , 53 , 45 , 52 , 59 , 49 , 50 , 43 , 49 , 50 , 57 , 46 , 57 , 43 , 46 , 48 , 54 , 58 , 42 , 51 , 27 , 32 , 46 , 40 , 46 , 54 , 51 , 50 , 50 , 51 , 53 , 55 , 52 , 58 , 57 , 55 , 50 , 58 , 59 , 46 , 51 , 59 , 62 , 56 , 26 , 24 , 51 , 50 , 28 , 40 , 47 , 69 , 53 , 60 , 58 , 54 , 47 , 58 , 61 , 51 , 73 , 60 , 70 , 72 , 68 , 70 , 62 , 70 , 65 , 69 , 60 , 57 , 56 , 60 , 62 , 65 , 74 , 59 , 68 , 40 , 58 , 58 , 57 , 61 , 68 , 69 , 66 , 64 , 67 , 67 , 70 , 67 , 74 , 72 , 69 , 61 , 69 , 67 , 54 , 62 , 69 , 66 , 66 , 31 , 32 , 53 , 46 , 31 , 41 , 49 , 53 , 53 , 52 , 51 , 53 , 46 , 49 , 54 , 49 , 60 , 54 , 53 , 65 , 58 , 54 , 53 , 57 , 60 , 61 , 54 , 46 , 53 , 50 , 56 , 49 , 62 , 51 , 60 , 42 , 70 , 55 , 52 , 46 , 53 , 53 , 56 , 59 , 56 , 60 , 59 , 52 , 61 , 49 , 60 , 54 , 58 , 54 , 53 , 54 , 59 , 58 , 57 , 2 , 1 , 1 , 1 , 2 , 0 , 3 , 1 , 2 , 1 , 2 , 0 , 4 , 2 , 3 , 3 , 2 , 1 , 6 , 1 , 1 , 2 , 3 , 3 , 1 , 4 , 3 , 1 , 2 , 4 , 4 , 3 , 1 , 3 , 3 , 1 , 2 , 3 , 0 , 3 , 4 , 4 , 7 , 1 , 3 , 2 , 4 , 3 , 4 , 5 , 3 , 4 , 4 , 7 , 3 , 8 , 7 , 7 , 5 , 2 , 2 , 5 , 2 , 2 , 4 , 2 , 2 , 2 , 2 , 3 , 1 , 4 , 6 , 4 , 4 , 4 , 3 , 3 , 4 , 2 , 3 , 3 , 1 , 1 , 4 , 3 , 5 , 3 , 6 , 5 , 5 , 2 , 2 , 3 , 3 , 6 , 4 , 2 , 4 , 5 , 5 , 7 , 3 , 5 , 5 , 6 , 3 , 4 , 5 , 4 , 5 , 4 , 9 , 2 , 4 , 10 , 5 , 7 , 61 , 59 , 48 , 53 , 41 , 59 , 58 , 24 , 53 , 42 , 56 , 57 , 58 , 61 , 54 , 54 , 45 , 47 , 50 , 50 , 43 , 51 , 41 , 56 , 39 , 47 , 51 , 59 , 56 , 59 , 40 , 48 , 56 , 51 , 53 , 57 , 52 , 55 , 47 , 52 , 52 , 63 , 55 , 30 , 54 , 47 , 51 , 53 , 42 , 44 , 44 , 52 , 44 , 43 , 43 , 61 , 54 , 22 , 50 , 1 , 2 , 3 , 5 , 6 , 9 , 10 , 11 , 12 , 13 , 16 , 17 , 18 , 19 , 21 , 22 , 23 , 24 , 25 , 26 , 27 , 28 , 29 , 32 , 33 , 34 , 35 , 36 , 37 , 38 , 40 , 41 , 42 , 43 , 44 , 45 , 46 , 47 , 49 , 51 , 52 , 53 , 55 , 56 , 57 , 58 , 59 , NA , Fold1",Fold1
"0 , 0 , 1 , 1 , 1 , 1 , 1 , 2 , 2 , 2 , 2 , 2 , 2 , 2 , 3 , 3 , 3 , 3 , 4 , 4 , 4 , 4 , 4 , 4 , 4 , 5 , 5 , 5 , 5 , 5 , 5 , 5 , 5 , 5 , 5 , 6 , 6 , 6 , 6 , 6 , 6 , 6 , 7 , 7 , 7 , 8 , 8 , 8 , 9 , 9 , 10 , 11 , 11 , 11 , 12 , 12 , 12 , 14 , 14 , 29 , 30 , 44 , 48 , 22 , 42 , 47 , 61 , 50 , 53 , 60 , 53 , 49 , 57 , 52 , 50 , 69 , 53 , 64 , 69 , 61 , 59 , 52 , 55 , 59 , 67 , 54 , 65 , 51 , 57 , 58 , 62 , 70 , 47 , 66 , 38 , 43 , 56 , 51 , 55 , 66 , 61 , 61 , 61 , 61 , 67 , 68 , 63 , 68 , 64 , 63 , 57 , 67 , 68 , 50 , 59 , 65 , 71 , 62 , 21 , 21 , 38 , 36 , 9 , 30 , 43 , 47 , 42 , 41 , 46 , 41 , 39 , 42 , 42 , 42 , 53 , 45 , 52 , 59 , 49 , 50 , 43 , 49 , 50 , 57 , 46 , 57 , 43 , 46 , 48 , 54 , 58 , 42 , 51 , 27 , 32 , 46 , 40 , 46 , 54 , 51 , 50 , 50 , 51 , 53 , 55 , 52 , 58 , 57 , 55 , 50 , 58 , 59 , 46 , 51 , 59 , 62 , 56 , 26 , 24 , 51 , 50 , 28 , 40 , 47 , 69 , 53 , 60 , 58 , 54 , 47 , 58 , 61 , 51 , 73 , 60 , 70 , 72 , 68 , 70 , 62 , 70 , 65 , 69 , 60 , 57 , 56 , 60 , 62 , 65 , 74 , 59 , 68 , 40 , 58 , 58 , 57 , 61 , 68 , 69 , 66 , 64 , 67 , 67 , 70 , 67 , 74 , 72 , 69 , 61 , 69 , 67 , 54 , 62 , 69 , 66 , 66 , 31 , 32 , 53 , 46 , 31 , 41 , 49 , 53 , 53 , 52 , 51 , 53 , 46 , 49 , 54 , 49 , 60 , 54 , 53 , 65 , 58 , 54 , 53 , 57 , 60 , 61 , 54 , 46 , 53 , 50 , 56 , 49 , 62 , 51 , 60 , 42 , 70 , 55 , 52 , 46 , 53 , 53 , 56 , 59 , 56 , 60 , 59 , 52 , 61 , 49 , 60 , 54 , 58 , 54 , 53 , 54 , 59 , 58 , 57 , 2 , 1 , 1 , 1 , 2 , 0 , 3 , 1 , 2 , 1 , 2 , 0 , 4 , 2 , 3 , 3 , 2 , 1 , 6 , 1 , 1 , 2 , 3 , 3 , 1 , 4 , 3 , 1 , 

#### 2. specifying the model and workflow

We use `tidymodels` syntax to specify our linear regression model, and put everything into a *workflow*: 

In [7]:
lm_mod = linear_reg() %>%
  set_engine("lm")

lm_wf <- 
  workflow() %>%
  add_model(lm_mod) %>%
  add_formula(Age ~ .)

#### 3. fitting the model on the resampled data 

Now we apply the workflow to the partitions of the data that we prepared before (our 5-fold partitions: 4 folds for training, 1 fold for validation, in turns $\rightarrow$ 5 different partitions of the data)

In [8]:
lm_fit <- 
  lm_wf %>% 
  fit_resamples(
    folds,
    control = control_resamples(save_pred = TRUE))

#### 4. evaluating the performance of the model

To evaluate the performance of the model we first collect the error metrics: RMSE and $R^2$. These are averages over the 5 validation folds, together with the standard error:

In [9]:
collect_metrics(lm_fit)

.metric,.estimator,mean,n,std_err,.config
<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>
rmse,standard,2.3965328,5,0.07435522,Preprocessor1_Model1
rsq,standard,0.5827837,5,0.05952589,Preprocessor1_Model1


In [10]:
metrs <- collect_metrics(lm_fit)
error_pct = metrs$mean[1]/mean(ch4$Age)
print(error_pct)

[1] 0.4404842


We see the error is **`r round(error_pct,2)*100`%** of the average age of bats in the dataset. 
We then look at predictions in the validation folds, and measure the Pearson linear correlation between predicted and observed age values:

In [11]:
collect_predictions(lm_fit) %>% group_by(id) %>% summarise(pearson = cor(.pred,Age))

id,pearson
<chr>,<dbl>
Fold1,0.6703022
Fold2,0.7446519
Fold3,0.6932762
Fold4,0.8796954
Fold5,0.8096982


### Repeated cross-validation

This was one single 5-fold cross-validation partition. However, to really take advantage of the full power of resampling methods and get robust estimates of the model performance, it is better (and usually done) to replicate the k-fold cross-validation **n times** (thereby resampling at each of the *n* replicates a different 5-fold partition):

In [12]:
folds = vfold_cv(ch4, v = 5, repeats = 10)

In [13]:
lm_fit <- 
  lm_wf %>% 
  fit_resamples(
    folds,
    control = control_resamples(save_pred = TRUE))

In [14]:
collect_metrics(lm_fit)

.metric,.estimator,mean,n,std_err,.config
<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>
rmse,standard,2.4450237,50,0.07434294,Preprocessor1_Model1
rsq,standard,0.5274649,50,0.02841381,Preprocessor1_Model1


We see that the standard errors of the estimated RMSE and $R^2$ are now smaller (averages of 50 values instead of 5 values!). We finally get the average Pearson correlation between predicted and observed ages, and its standard deviation:

In [15]:
collect_predictions(lm_fit) %>% group_by(id) %>% summarise(pearson = cor(.pred,Age)) %>% summarise(avg = mean(pearson), std = sd(pearson))

avg,std
<dbl>,<dbl>
0.7134351,0.02715769


## Classification

We now see how we can apply `tidymodels` to a classification problem.
We use our `dogs` dataset (reduced: only 45 SNPs):

In [ ]:
dogs <- fread("../data/dogs_imputed_reduced.raw")
dogs <- dogs %>%
  select(-c(IID,FID,PAT,MAT,SEX))

## values for logistic regression in glm() must be 0 - 1
dogs$PHENOTYPE = dogs$PHENOTYPE-1
head(dogs)
print(nrow(dogs))
print(ncol(dogs)-1)

This dataset is unbalanced: we have far fewer cases than controls:

In [ ]:
dogs %>%
  group_by(PHENOTYPE) %>%
  summarise(N=n())

When data are unbalanced, one possible approach to improve the performance of the predictive model is to **subsample** the data:

- **downsampling** or **undersampling**: *sample down* the *majority* class until it has the same frequency of the *minority* class
- **oversample** the data: generate new samples of the *minority* class based on its *neighbourhood* (determined by a similarity matrix)

Subsampling usually improves model performance (see for instance [Menardi and Torelli, 2012](https://link.springer.com/article/10.1007/s10618-012-0295-5)).

It is important that subsampling is **applied only to the training data** (e.g. when doing k-fold cross-validation, under- or over-sampling must happen only in the k-1 folds used for training).

#### 1. subsampling the data

In this case, data will be oversampled (the dataset is small):

In [ ]:
library("themis")
imbal_rec <- 
  recipe(PHENOTYPE ~ ., data = dogs) %>%
  step_bin2factor(PHENOTYPE) %>%
  step_rose(PHENOTYPE, skip = TRUE) ## skip == TRUE is important to make sure that subsampling happens only in the training data

In [ ]:
prep(imbal_rec) %>% juice() %>% group_by(PHENOTYPE) %>% summarise(N=n())

#### 2. partitioning the data

In [ ]:
folds <- vfold_cv(dogs, v = 5, strata = "PHENOTYPE", repeats = 5)

#### 3. specifying the logistic regression model and workflow

In [ ]:
log_mod = logistic_reg(
  mode = "classification",
  engine = "glm"
)

log_wf <- 
  workflow() %>%
  add_model(log_mod) %>%
  add_recipe(imbal_rec)

#### 4. defining performance metrics

In [ ]:
cls_metrics <- metric_set(roc_auc,sens,spec)

#### 5. fitting the model on the resampled data 

In [ ]:
log_res <- fit_resamples(
  log_wf, 
  resamples = folds, 
  metrics = cls_metrics,
  control = control_resamples(save_pred = TRUE)
)

#### 6. evaluating the performance of the model

We look at the average:

- `AUC`: area under the ROC curve
- `sens`: TPR (true positive rate)
- `spec`: TNR (true negative rate)

In [ ]:
collect_metrics(log_res)

Then we can look at predictions and calculate the overall error rate:

In [ ]:
collect_predictions(log_res) %>% group_by(id,id2) %>% summarise(errors = sum(.pred_class != PHENOTYPE), tot = length(PHENOTYPE), error_rate = errors/tot)

In [ ]:
collect_predictions(log_res) %>% 
  group_by(id,id2) %>% 
  summarise(errors = sum(.pred_class != PHENOTYPE), tot = length(PHENOTYPE), error_rate = errors/tot) %>%
  ungroup() %>%
  summarise(avg_er = mean(error_rate), std = sd(error_rate))